In this notebook, I will...
   - calculate baseline error
   - run random forest regressors on abundance 
   - load regional dfs add island col as ID, rbind all 5 dfs 
   - predict out for each group 

#### Scoring models

We evaluated the performance of the abundance models based on mean average error (accuracy) and mean absolute percent error (MAPE).

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, confusion_matrix, auc
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import *
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report
import pickle
import requests
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import utm

In [2]:
os.chdir('C:/Users/linds/OneDrive/Documents/samoa_corals_data')

## Import data

In [3]:
coral_types=['scler','branching','columnar','encrusting','free_livin','massive','plate']
target_types=['binary','percent']
df = dict()

for i in range(0,len(coral_types)):
    for j in range(0,len(target_types)):
        df[str(coral_types[i])+'_'+str(target_types[j])]=pd.read_csv(str(coral_types[i])+'_'+str(target_types[j])+'.csv')
        del df[str(coral_types[i])+'_'+str(target_types[j])]['Unnamed: 0'] # artifact indexing column
# Access the data as, e.g., df['scler_percent']

# Scler (all corals)

### Train/test split, baseline error

The baseline is the estimate I would get if I simply predicted the average abundance across all cells. If I can improve upon this by using my model, then my approach is valid.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df['scler_percent'].drop(['Sclr_Rw','lat','lon','ID'], axis=1), 
                                                    df['scler_percent']['Sclr_Rw'], 
                                                    test_size = 0.3, random_state = 30)


print('Scler Training Features Shape:', X_train.shape)
print('Scler Training Labels Shape:', y_train.shape)
print('Scler Testing Features Shape:', X_test.shape)
print('Scler Testing Labels Shape:', y_test.shape)

# The baseline predictions are the averages

baseline_preds = np.array([y_train.mean()] * len(y_train))
baseline_errors = abs(baseline_preds - y_train)
print('Baseline prediction error: ', round(np.mean(baseline_errors), 2))

Scler Training Features Shape: (1372, 9)
Scler Training Labels Shape: (1372,)
Scler Testing Features Shape: (588, 9)
Scler Testing Labels Shape: (588,)
Baseline prediction error:  14.31


### Train random forest regressor

In [5]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 30)

# Train the model on training data
rf.fit(X_train, y_train)

# Predicting to the test data
predictions = rf.predict(X_test)

# Calculate the absolute errors
errors = abs(predictions - y_test)

# Print out the mean absolute error (mae)
print('Mean Absolute Error: ', round(np.mean(errors), 2))

Mean Absolute Error:  10.22


### Tune the model

In [6]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
bootstrap = [True, False] # Create the random grid

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'bootstrap': bootstrap}

print(random_grid)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()

# Random search of parameters, using 10 fold cross validation, 
# Search across n_iter * cv different combinations, and use all available cores
rf_best = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 40, cv = 10, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rf_best.fit(X_train, y_train)
print(rf_best.best_params_)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'bootstrap': [True, False]}
Fitting 10 folds for each of 40 candidates, totalling 400 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed:  6.0min finished


{'n_estimators': 600, 'max_features': 'sqrt', 'bootstrap': True}


In [7]:
#Predicting on the test set
predictions = rf_best.predict(X_test)

# Calculate the absolute errors
errors = abs(predictions - y_test)

# Print out the mean absolute error (mae)
print('Mean Absolute Error: ', round(np.mean(errors), 2))

Mean Absolute Error:  10.15


In [5]:
# Retrain with the optimal parameters
rf = RandomForestRegressor(n_estimators = 600, max_features = 'sqrt', bootstrap = True, random_state = 30)

# Train the model on training data
rf.fit(X_train, y_train)

# Predicting to the test data
predictions = rf.predict(X_test)

# Calculate the absolute errors
errors = abs(predictions - y_test)

# Print out the mean absolute error (mae)
print('Mean Absolute Error: ', round(np.mean(errors), 2))

# Get numerical feature importances
feature_list = list(X_train.columns)
importances = list(rf.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
print(feature_importances)

# Save the test data (actual) and predictions to df for visualizations later
scler_preds = pd.DataFrame(data = {'actual': y_test, 'prediction': predictions})
scler_preds.to_csv('scler_preds.csv', index=False)

Mean Absolute Error:  10.1
[('bty', 0.17), ('rug', 0.14), ('slope', 0.13), ('shore_ed', 0.13), ('vill_ed', 0.13), ('asp', 0.12), ('hs', 0.08), ('ucur', 0.06), ('vcur', 0.05)]


Using the random forest regressor to predict percent abundance of all scleractinian corals across all sample sites reduced predictive error by nearly one-third (from 14.31% to 10.22%) compared to the application of a "baseline average" for prediction (i.e., the calculation of a uniform basic average abundance of coral across the sample sites). We used 10-fold cross validation over 40 iterations to tune the models, which further reduced our error by X% - Y% for each model (insert table reference here).

### Pickle the model

In [16]:
#with open('scler_abundance_model.pkl', 'wb') as fid:
#    pickle.dump(rf, fid, 2)

### Load regional dfs

Adding an island ID column, concatenating the dfs...

In [7]:
os.chdir('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tau')
tau_bty_pts = pd.read_csv('tau_bty_pts.csv')
tau_bty_pts.rename(columns={'tau_dbmb_mos': 'bty'}, inplace=True)
tau_bty_pts['ID'] = 'Tau'

os.chdir('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Tut')
tut_bty_pts = pd.read_csv('tut_bty_pts.csv')
tut_bty_pts.rename(columns={'tut_dbmb': 'bty'}, inplace=True)
tut_bty_pts['ID'] = 'Tut'

os.chdir('C:/Users/linds/OneDrive/Documents/samoa_corals_data/OfuOlo')
oo_bty_pts = pd.read_csv('oo_bty_pts.csv')
oo_bty_pts.rename(columns={'depth': 'bty'}, inplace=True)
oo_bty_pts['ID'] = 'OfuOlo'

os.chdir('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Rose')
rose_bty_pts = pd.read_csv('rose_bty_pts.csv')
rose_bty_pts.rename(columns={'rose_5m_dbmb': 'bty'}, inplace=True)
rose_bty_pts['ID'] = 'Rose'

os.chdir('C:/Users/linds/OneDrive/Documents/samoa_corals_data/Swains')
swains_bty_pts = pd.read_csv('swains_bty_pts.csv')
swains_bty_pts.rename(columns={'swa_dbmb_5m': 'bty'}, inplace=True)
swains_bty_pts['ID'] = 'Swains'

In [45]:
os.chdir('C:/Users/linds/OneDrive/Documents/samoa_corals_data/')
AS_all = pd.concat([tau_bty_pts, tut_bty_pts, oo_bty_pts,
                   rose_bty_pts, swains_bty_pts])

del AS_all['Unnamed: 0']
del AS_all['dummy']
print(AS_all.shape)
AS_all.columns
AS_all.bty = AS_all.bty * -1

AS_all.head()
AS_all.to_csv('AS_all.csv', index=False)

(146074, 16)


### Predict out to American Samoa concatenated df

In [16]:
AS_all = pd.read_csv('AS_all.csv')
AS_all.dropna(inplace=True)
print(AS_all.shape)
AS_all.head(3)

(144990, 16)


,bty,hs,rug,slope,asp,shore_ed,wl_ed,vill_third_q,vill_ed,esi,esi_ed,ucur,vcur,x,y,ID
674,197.610001,1.959184,1.843792,40.829605,259.345642,784.577155,6325.764809,287.994,2487.227411,Freshwater Marshes,18418.813311,-0.00489,-0.01047,666925.0,8429050.0,Tau
675,198.049805,1.979381,1.600071,40.400249,144.983948,790.916596,6303.516019,287.994,2460.637660,Freshwater Marshes,18468.167397,-0.00489,-0.01047,666975.0,8429050.0,Tau
676,198.599426,1.989796,1.541323,43.999477,198.824860,799.507074,6281.586422,287.994,2434.784425,Freshwater Marshes,18517.524949,-0.00489,-0.01047,667025.0,8429050.0,Tau


In [23]:
AS_all.to_csv('AS_all.csv', index=False)

In [18]:
# scler: bty, hs, rug, slope, asp, shore_ed, vill_ed, ucur, vcur
scler_pred_out = AS_all[['bty', 'hs', 'rug', 'slope', 'asp', 'shore_ed', 'vill_ed', 'ucur', 'vcur']]
scler_predictions = rf.predict(scler_pred_out)

In [31]:
print(y_train.mean())
print(scler_predictions.mean())
AS_all['scler_preds'] = scler_predictions
AS_all.to_csv('AS_all.csv', index=False)

8.922521865889212
11.22077150234499


In [3]:
AS_all = pd.read_csv('AS_all.csv')

In [5]:
AS_all.shape

(144990, 21)

In [6]:
AS_all.columns

Index(['bty', 'hs', 'rug', 'slope', 'asp', 'shore_ed', 'wl_ed', 'vill_third_q',
       'vill_ed', 'esi', 'esi_ed', 'ucur', 'vcur', 'x', 'y', 'ID',
       'scler_preds', 'plate_preds', 'massive_preds', 'free_living_preds',
       'encrusting_preds'],
      dtype='object')